In [1]:
import numpy as np
import math
import time
import torch
import os
from torch.autograd import Variable
import matplotlib.pyplot as plt
from Modules.training_functions import *
from Modules.pennylane_functions import *

# if gpu available, set device to gpu
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using the GPU")
else:
    device = torch.device("cpu")
    print("WARNING: Could not find GPU, using the CPU")

T=10
# load dataset
mnist_images0 = np.load(f'Data/dataset_ld_{ld_dim}_0.npy')
mnist_images1 = np.load(f'Data/dataset_ld_{ld_dim}_1.npy')

mnist_images =np.concatenate((mnist_images0[:1000], mnist_images1[:1000]), axis = 0)
print(np.shape(mnist_images))

np.random.shuffle(mnist_images)
mnist_images = torch.tensor(mnist_images).to(device)

# make dataloader
data_loader = torch.utils.data.DataLoader(mnist_images, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
qc_array=np.array([0,16,24,28])
min_array=np.array([0.1,0.05,0.01,0.08])
layer_array=np.array([5,10,20,50])
print(NUM_QUBITS)
print(T)
for layer_indx in range(len(layer_array)):
    n_layer=layer_array[layer_indx]
    for q_indx in range(len(qc_array)):
        qc=qc_array[q_indx]
        for min_indx in range(len(min_array)):
            min_b=min_array[min_indx]

            betas      = np.insert(np.linspace(10e-8,min_b, T), 0, 0)
            print(np.shape(betas))
            alphas     = 1 - betas
            alphas_bar = np.cumprod(alphas)
            pi         = math.pi
            betas      = torch.tensor(betas).float().to(device)
            alphas     = torch.tensor(alphas).float().to(device)
            alphas_bar = torch.tensor(alphas_bar).float().to(device)
            theta_1    = Variable(torch.rand((n_layer*3*NUM_QUBITS+n_layer*3*(NUM_QUBITS)), device = device), requires_grad=True)
            optimizer = torch.optim.Adam([theta_1], lr = LEARNING_RATE)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = SCHEDULER_PATIENCE, gamma = SCHEDULER_GAMMA, verbose = False)
            trained_thetas_1 = []
            loss_history = []
            best_loss = 1e10

            for epoch in range(NUM_EPOCHS):
                print(epoch)

                t0 = time.time()
                num_batch=0
                tot_loss=0

                for image_batch in data_loader:

                    # extract batch of random times and betas
                    t = torch.randint(0, T, size = (BATCH_SIZE, ), device=device)
                    betas_batch = betas[t].to(device)
                    alphas_batch=alphas_bar[t].to(device)

                    # assemble input at t add noise (t+1)
                    target_batch = assemble_input(image_batch, t, alphas_bar,ld_dim ,device)
                    input_batch  = noise_step(target_batch, t+1, betas,ld_dim, device)
                    target_batch = target_batch / torch.norm(target_batch, dim = 1).view(-1, 1)
                    input_batch  = input_batch / torch.norm(input_batch, dim = 1).view(-1, 1)
                    zero = torch.zeros(BATCH_SIZE, ld_dim).to(device)

                    # concatenate the two tensors along the second dimension
                    input_batch = torch.cat((input_batch, zero), dim=1)
                    target_batch = torch.cat((target_batch, zero), dim=1)
                    # Feed to circuit, compute the loss and update the weights
                    num_batch+=1
                    loss = loss_fn_aq(qc,theta_1,n_layer, input_batch, target_batch)
                    tot_loss+=loss.item()
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

                # append parameters and print loss
                trained_thetas_1.append(theta_1.cpu().clone().detach().numpy())

                loss_history.append(tot_loss/num_batch)
                if loss.item()< best_loss:
                    best_loss=loss.item()

                # implement learning rate scheduler
                scheduler.step()


            # print every epoch
                print(f'{qc} T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={((time.time()-t0)*(NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))/60:.2f} min{min_b} nl{n_layer}')
                #print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={(((NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))):.2f} min{min_b} nl{n_layer}')
                
            np.save(f'thetas_T{T}_nl{n_layer}_min{min_b}_qc{qc}_{Q_ANCILLA}_ld{ld_dim}.npy',trained_thetas_1)
            np.save(f'loss__T{T}_nl{n_layer}_min{min_b}_qc{qc}_ancilla{Q_ANCILLA}_ld{ld_dim}.npy',loss_history)

(2000, 16)
5
10
(11,)
0
0 T=10 Epoch: 1/30 - Loss: 0.9033 b_loss=0.9033 - T: 1.14s/epoch ,tempo_previto=36.62 min0.1 nl5
1
0 T=10 Epoch: 2/30 - Loss: 0.8185 b_loss=0.8185 - T: 1.17s/epoch ,tempo_previto=37.33 min0.1 nl5
2
0 T=10 Epoch: 3/30 - Loss: 0.7126 b_loss=0.7126 - T: 1.23s/epoch ,tempo_previto=39.23 min0.1 nl5
3
0 T=10 Epoch: 4/30 - Loss: 0.6608 b_loss=0.6608 - T: 1.21s/epoch ,tempo_previto=38.73 min0.1 nl5
4
0 T=10 Epoch: 5/30 - Loss: 0.5953 b_loss=0.5953 - T: 1.24s/epoch ,tempo_previto=39.44 min0.1 nl5
5
0 T=10 Epoch: 6/30 - Loss: 0.5378 b_loss=0.5378 - T: 1.26s/epoch ,tempo_previto=40.18 min0.1 nl5
6
0 T=10 Epoch: 7/30 - Loss: 0.5458 b_loss=0.5378 - T: 1.36s/epoch ,tempo_previto=43.28 min0.1 nl5
7
0 T=10 Epoch: 8/30 - Loss: 0.5445 b_loss=0.5378 - T: 1.13s/epoch ,tempo_previto=35.91 min0.1 nl5
8
0 T=10 Epoch: 9/30 - Loss: 0.4773 b_loss=0.4773 - T: 1.14s/epoch ,tempo_previto=36.37 min0.1 nl5
9
0 T=10 Epoch: 10/30 - Loss: 0.5084 b_loss=0.4773 - T: 1.25s/epoch ,tempo_previto=39.9

0 T=10 Epoch: 22/30 - Loss: 0.1713 b_loss=0.1713 - T: 1.24s/epoch ,tempo_previto=34.23 min0.01 nl5
22
0 T=10 Epoch: 23/30 - Loss: 0.1724 b_loss=0.1713 - T: 1.21s/epoch ,tempo_previto=33.30 min0.01 nl5
23
0 T=10 Epoch: 24/30 - Loss: 0.1665 b_loss=0.1665 - T: 1.26s/epoch ,tempo_previto=34.74 min0.01 nl5
24
0 T=10 Epoch: 25/30 - Loss: 0.1649 b_loss=0.1649 - T: 1.18s/epoch ,tempo_previto=32.68 min0.01 nl5
25
0 T=10 Epoch: 26/30 - Loss: 0.1663 b_loss=0.1649 - T: 1.24s/epoch ,tempo_previto=34.26 min0.01 nl5
26
0 T=10 Epoch: 27/30 - Loss: 0.1698 b_loss=0.1649 - T: 1.26s/epoch ,tempo_previto=34.68 min0.01 nl5
27
0 T=10 Epoch: 28/30 - Loss: 0.1719 b_loss=0.1649 - T: 1.26s/epoch ,tempo_previto=34.74 min0.01 nl5
28
0 T=10 Epoch: 29/30 - Loss: 0.1621 b_loss=0.1621 - T: 1.26s/epoch ,tempo_previto=34.58 min0.01 nl5
29
0 T=10 Epoch: 30/30 - Loss: 0.1693 b_loss=0.1621 - T: 1.25s/epoch ,tempo_previto=34.48 min0.01 nl5
(11,)
0
0 T=10 Epoch: 1/30 - Loss: 0.9126 b_loss=0.9126 - T: 1.20s/epoch ,tempo_previ

16 T=10 Epoch: 13/30 - Loss: 0.3456 b_loss=0.3297 - T: 1.13s/epoch ,tempo_previto=33.15 min0.05 nl5
13
16 T=10 Epoch: 14/30 - Loss: 0.3457 b_loss=0.3297 - T: 1.29s/epoch ,tempo_previto=37.65 min0.05 nl5
14
16 T=10 Epoch: 15/30 - Loss: 0.3296 b_loss=0.3296 - T: 1.21s/epoch ,tempo_previto=35.39 min0.05 nl5
15
16 T=10 Epoch: 16/30 - Loss: 0.3224 b_loss=0.3224 - T: 1.24s/epoch ,tempo_previto=36.18 min0.05 nl5
16
16 T=10 Epoch: 17/30 - Loss: 0.3367 b_loss=0.3224 - T: 1.24s/epoch ,tempo_previto=36.28 min0.05 nl5
17
16 T=10 Epoch: 18/30 - Loss: 0.3376 b_loss=0.3224 - T: 1.27s/epoch ,tempo_previto=37.19 min0.05 nl5
18
16 T=10 Epoch: 19/30 - Loss: 0.3269 b_loss=0.3224 - T: 1.27s/epoch ,tempo_previto=36.99 min0.05 nl5
19
16 T=10 Epoch: 20/30 - Loss: 0.3033 b_loss=0.3033 - T: 1.28s/epoch ,tempo_previto=37.23 min0.05 nl5
20
16 T=10 Epoch: 21/30 - Loss: 0.3284 b_loss=0.3033 - T: 1.29s/epoch ,tempo_previto=37.74 min0.05 nl5
21
16 T=10 Epoch: 22/30 - Loss: 0.3175 b_loss=0.3033 - T: 1.16s/epoch ,tempo

24 T=10 Epoch: 3/30 - Loss: 0.7324 b_loss=0.7324 - T: 1.27s/epoch ,tempo_previto=39.33 min0.1 nl5
3
24 T=10 Epoch: 4/30 - Loss: 0.6634 b_loss=0.6634 - T: 1.29s/epoch ,tempo_previto=39.83 min0.1 nl5
4
24 T=10 Epoch: 5/30 - Loss: 0.6062 b_loss=0.6062 - T: 1.31s/epoch ,tempo_previto=40.45 min0.1 nl5
5
24 T=10 Epoch: 6/30 - Loss: 0.5940 b_loss=0.5940 - T: 1.19s/epoch ,tempo_previto=36.70 min0.1 nl5
6
24 T=10 Epoch: 7/30 - Loss: 0.5718 b_loss=0.5718 - T: 1.16s/epoch ,tempo_previto=35.76 min0.1 nl5
7
24 T=10 Epoch: 8/30 - Loss: 0.5633 b_loss=0.5633 - T: 1.31s/epoch ,tempo_previto=40.48 min0.1 nl5
8
24 T=10 Epoch: 9/30 - Loss: 0.5163 b_loss=0.5163 - T: 1.19s/epoch ,tempo_previto=36.83 min0.1 nl5
9
24 T=10 Epoch: 10/30 - Loss: 0.4823 b_loss=0.4823 - T: 1.32s/epoch ,tempo_previto=40.81 min0.1 nl5
10
24 T=10 Epoch: 11/30 - Loss: 0.4994 b_loss=0.4823 - T: 1.28s/epoch ,tempo_previto=39.37 min0.1 nl5
11
24 T=10 Epoch: 12/30 - Loss: 0.4822 b_loss=0.4822 - T: 1.33s/epoch ,tempo_previto=40.82 min0.1 n

24 T=10 Epoch: 24/30 - Loss: 0.1846 b_loss=0.1846 - T: 1.17s/epoch ,tempo_previto=31.12 min0.01 nl5
24
24 T=10 Epoch: 25/30 - Loss: 0.1863 b_loss=0.1846 - T: 1.26s/epoch ,tempo_previto=33.62 min0.01 nl5
25
24 T=10 Epoch: 26/30 - Loss: 0.1817 b_loss=0.1817 - T: 1.16s/epoch ,tempo_previto=30.81 min0.01 nl5
26
24 T=10 Epoch: 27/30 - Loss: 0.1898 b_loss=0.1817 - T: 1.19s/epoch ,tempo_previto=31.64 min0.01 nl5
27
24 T=10 Epoch: 28/30 - Loss: 0.1927 b_loss=0.1817 - T: 1.28s/epoch ,tempo_previto=33.87 min0.01 nl5
28
24 T=10 Epoch: 29/30 - Loss: 0.1854 b_loss=0.1817 - T: 1.21s/epoch ,tempo_previto=32.05 min0.01 nl5
29
24 T=10 Epoch: 30/30 - Loss: 0.1896 b_loss=0.1817 - T: 1.32s/epoch ,tempo_previto=34.91 min0.01 nl5
(11,)
0
24 T=10 Epoch: 1/30 - Loss: 0.8830 b_loss=0.8830 - T: 1.28s/epoch ,tempo_previto=32.04 min0.08 nl5
1
24 T=10 Epoch: 2/30 - Loss: 0.8125 b_loss=0.8125 - T: 1.22s/epoch ,tempo_previto=30.35 min0.08 nl5
2
24 T=10 Epoch: 3/30 - Loss: 0.7100 b_loss=0.7100 - T: 1.16s/epoch ,tempo

...

0 T=10 Epoch: 27/30 - Loss: 0.0132 b_loss=0.0129 - T: 12.50s/epoch ,tempo_previto=44.36 min0.01 nl50
27
0 T=10 Epoch: 28/30 - Loss: 0.0145 b_loss=0.0129 - T: 12.59s/epoch ,tempo_previto=44.49 min0.01 nl50
28
0 T=10 Epoch: 29/30 - Loss: 0.0133 b_loss=0.0129 - T: 13.01s/epoch ,tempo_previto=45.75 min0.01 nl50
29
0 T=10 Epoch: 30/30 - Loss: 0.0129 b_loss=0.0129 - T: 12.87s/epoch ,tempo_previto=45.03 min0.01 nl50
(11,)
0
0 T=10 Epoch: 1/30 - Loss: 0.4473 b_loss=0.4473 - T: 12.74s/epoch ,tempo_previto=25.27 min0.08 nl50
1
0 T=10 Epoch: 2/30 - Loss: 0.2911 b_loss=0.2911 - T: 13.10s/epoch ,tempo_previto=25.77 min0.08 nl50
2
0 T=10 Epoch: 3/30 - Loss: 0.1963 b_loss=0.1963 - T: 12.34s/epoch ,tempo_previto=24.06 min0.08 nl50
3
0 T=10 Epoch: 4/30 - Loss: 0.1218 b_loss=0.1218 - T: 12.77s/epoch ,tempo_previto=24.68 min0.08 nl50
4
0 T=10 Epoch: 5/30 - Loss: 0.0824 b_loss=0.0824 - T: 13.04s/epoch ,tempo_previto=24.99 min0.08 nl50
5
0 T=10 Epoch: 6/30 - Loss: 0.0611 b_loss=0.0611 - T: 13.10s/epoch ,te

16 T=10 Epoch: 16/30 - Loss: 0.0476 b_loss=0.0429 - T: 15.50s/epoch ,tempo_previto=81.11 min0.05 nl50
16
16 T=10 Epoch: 17/30 - Loss: 0.0428 b_loss=0.0428 - T: 13.99s/epoch ,tempo_previto=72.97 min0.05 nl50
17
16 T=10 Epoch: 18/30 - Loss: 0.0452 b_loss=0.0428 - T: 15.15s/epoch ,tempo_previto=78.76 min0.05 nl50
18
16 T=10 Epoch: 19/30 - Loss: 0.0466 b_loss=0.0428 - T: 16.07s/epoch ,tempo_previto=83.30 min0.05 nl50
19
16 T=10 Epoch: 20/30 - Loss: 0.0456 b_loss=0.0428 - T: 15.34s/epoch ,tempo_previto=79.27 min0.05 nl50
20
16 T=10 Epoch: 21/30 - Loss: 0.0437 b_loss=0.0428 - T: 16.85s/epoch ,tempo_previto=86.77 min0.05 nl50
21
16 T=10 Epoch: 22/30 - Loss: 0.0460 b_loss=0.0428 - T: 16.23s/epoch ,tempo_previto=83.29 min0.05 nl50
22
16 T=10 Epoch: 23/30 - Loss: 0.0418 b_loss=0.0418 - T: 14.96s/epoch ,tempo_previto=76.53 min0.05 nl50
23
16 T=10 Epoch: 24/30 - Loss: 0.0430 b_loss=0.0418 - T: 14.63s/epoch ,tempo_previto=74.61 min0.05 nl50
24
16 T=10 Epoch: 25/30 - Loss: 0.0421 b_loss=0.0418 - T: 

24 T=10 Epoch: 5/30 - Loss: 0.2298 b_loss=0.2298 - T: 14.33s/epoch ,tempo_previto=99.14 min0.1 nl50
5
24 T=10 Epoch: 6/30 - Loss: 0.2178 b_loss=0.2178 - T: 14.64s/epoch ,tempo_previto=100.98 min0.1 nl50
6
24 T=10 Epoch: 7/30 - Loss: 0.2167 b_loss=0.2167 - T: 16.46s/epoch ,tempo_previto=113.29 min0.1 nl50
7
24 T=10 Epoch: 8/30 - Loss: 0.2163 b_loss=0.2163 - T: 16.39s/epoch ,tempo_previto=112.51 min0.1 nl50
8
24 T=10 Epoch: 9/30 - Loss: 0.2353 b_loss=0.2163 - T: 15.08s/epoch ,tempo_previto=103.28 min0.1 nl50
9
24 T=10 Epoch: 10/30 - Loss: 0.2198 b_loss=0.2163 - T: 14.30s/epoch ,tempo_previto=97.70 min0.1 nl50
10
24 T=10 Epoch: 11/30 - Loss: 0.2083 b_loss=0.2083 - T: 16.13s/epoch ,tempo_previto=109.95 min0.1 nl50
11
24 T=10 Epoch: 12/30 - Loss: 0.2057 b_loss=0.2057 - T: 15.41s/epoch ,tempo_previto=104.77 min0.1 nl50
12
24 T=10 Epoch: 13/30 - Loss: 0.2300 b_loss=0.2057 - T: 14.84s/epoch ,tempo_previto=100.64 min0.1 nl50
13
24 T=10 Epoch: 14/30 - Loss: 0.2132 b_loss=0.2057 - T: 13.82s/epoch

24 T=10 Epoch: 24/30 - Loss: 0.0412 b_loss=0.0392 - T: 13.13s/epoch ,tempo_previto=34.13 min0.01 nl50
24
24 T=10 Epoch: 25/30 - Loss: 0.0411 b_loss=0.0392 - T: 12.98s/epoch ,tempo_previto=33.53 min0.01 nl50
25
24 T=10 Epoch: 26/30 - Loss: 0.0398 b_loss=0.0392 - T: 12.76s/epoch ,tempo_previto=32.75 min0.01 nl50
26
24 T=10 Epoch: 27/30 - Loss: 0.0447 b_loss=0.0392 - T: 13.03s/epoch ,tempo_previto=33.22 min0.01 nl50
27
24 T=10 Epoch: 28/30 - Loss: 0.0407 b_loss=0.0392 - T: 13.18s/epoch ,tempo_previto=33.39 min0.01 nl50
28
24 T=10 Epoch: 29/30 - Loss: 0.0375 b_loss=0.0375 - T: 13.00s/epoch ,tempo_previto=32.72 min0.01 nl50
29
24 T=10 Epoch: 30/30 - Loss: 0.0439 b_loss=0.0375 - T: 12.47s/epoch ,tempo_previto=31.18 min0.01 nl50
(11,)
0
24 T=10 Epoch: 1/30 - Loss: 0.4315 b_loss=0.4315 - T: 12.96s/epoch ,tempo_previto=12.74 min0.08 nl50
1
24 T=10 Epoch: 2/30 - Loss: 0.3272 b_loss=0.3272 - T: 13.10s/epoch ,tempo_previto=12.67 min0.08 nl50
2
24 T=10 Epoch: 3/30 - Loss: 0.2616 b_loss=0.2616 - T: 

28 T=10 Epoch: 13/30 - Loss: 0.1627 b_loss=0.1505 - T: 12.79s/epoch ,tempo_previto=54.80 min0.05 nl50
13
28 T=10 Epoch: 14/30 - Loss: 0.1672 b_loss=0.1505 - T: 13.18s/epoch ,tempo_previto=56.25 min0.05 nl50
14
28 T=10 Epoch: 15/30 - Loss: 0.1855 b_loss=0.1505 - T: 12.43s/epoch ,tempo_previto=52.85 min0.05 nl50
15
28 T=10 Epoch: 16/30 - Loss: 0.1817 b_loss=0.1505 - T: 12.67s/epoch ,tempo_previto=53.63 min0.05 nl50
16
28 T=10 Epoch: 17/30 - Loss: 0.1638 b_loss=0.1505 - T: 13.25s/epoch ,tempo_previto=55.88 min0.05 nl50
17
28 T=10 Epoch: 18/30 - Loss: 0.1750 b_loss=0.1505 - T: 12.96s/epoch ,tempo_previto=54.43 min0.05 nl50
18
28 T=10 Epoch: 19/30 - Loss: 0.1569 b_loss=0.1505 - T: 13.00s/epoch ,tempo_previto=54.40 min0.05 nl50
19
28 T=10 Epoch: 20/30 - Loss: 0.1698 b_loss=0.1505 - T: 12.85s/epoch ,tempo_previto=53.54 min0.05 nl50
20
28 T=10 Epoch: 21/30 - Loss: 0.1603 b_loss=0.1505 - T: 12.85s/epoch ,tempo_previto=53.31 min0.05 nl50
21
28 T=10 Epoch: 22/30 - Loss: 0.1660 b_loss=0.1505 - T: 